##                                                    CCC - NLP to SQL generator

### Importing all the libraries

In [59]:
from dotenv import load_dotenv
import openai
import os
import langchain.llms as llms
from langchain.chains import create_sql_query_chain
import pandas as pd
import sqlite3
from langchain_community.utilities import SQLDatabase
from langchain.prompts import PromptTemplate
import openpyxl
from langchain.sql_database import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [60]:
working_dir = "C:/Users/nithi/OneDrive - University of Illinois Chicago/Documents/Capstone project/LLM project"
os.chdir(working_dir)

print(f"The current working directory: {os.getcwd()}")

The current working directory: C:\Users\nithi\OneDrive - University of Illinois Chicago\Documents\Capstone project\LLM project


### Importing and loading the csv file to SQLite Database

In [63]:
# Load CSV file
csv_file = 'dataset_flattened.csv'
df = pd.read_csv(csv_file)  

# Save to SQLite DB
db_file = 'dataset.db'  
conn = sqlite3.connect(db_file)
table_name = 'models'
df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()

print(f"Data from '{csv_file}' has been successfully saved to '{db_file}' in the table '{table_name}'.")


Data from 'dataset_flattened.csv' has been successfully saved to 'dataset.db' in the table 'models'.


In [96]:
# SQLite database connection
db = SQLDatabase.from_uri("sqlite:///dataset.db")

# List all the available table names in the database
print("Table Names:", db.get_usable_table_names())

# Execute the query
db.run("SELECT count(*) from models")


Table Names: ['models']


'[(4797,)]'

### Developing a chain with sample example

In [97]:
# GPT connection
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

#First Chain to take the user input and convert it to a SQL query
write_query = create_sql_query_chain(llm, db)
response = write_query.invoke({"question": "What is the "})
response 

'SELECT DISTINCT "Company_Name" \nFROM models\nORDER BY "Company_Name"\nLIMIT 5;'

In [71]:
#Second chain to excute the SQL queries
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | execute_query
chain.invoke({"question": "Different companies in our table"})


"[('Allstate',), ('Liberty Mutual',), ('Nationwide',), ('State Farm',)]"

###  Fine tuning the model with basic prompt engineering for our use case

In [99]:
# Prompt to generate SQL queries
SQl_prompt = PromptTemplate.from_template(
    """You are a SQLite expert. Given an input question, create a syntactically correct SQLite query to run, to answer the input question.
Remember, your query should aim to answer the question with a single SQL statement and limit the results appropriately. You can order the results to return the most informative data in the database.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
Never query for all columns from a table. Only query the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Be careful not to query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use the date('now') function to get the current date if the question involves "today".

Only use the following tables:

{table_info}.

Question: {input}
Provide up to {top_k} SQL variations."""
)

# Prompt to answer the questions
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.
    
Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

In [100]:
# Initialize Chain 1: Generate SQL Query
generate_sql_chain = create_sql_query_chain(prompt=SQl_prompt, llm=llm, db=db)

# Initialize Chain 2: Execute SQL Query and generate structured answer
execute_sql_chain = answer_prompt | llm | StrOutputParser()


def execute_combined_chain(question):
    # Step 1: Generate SQL Query
    sql_query = generate_sql_chain.invoke({"question": question, "top_k": 1})
    
    # Step 2: Execute the SQL Query to get the result
    sql_result = execute_query(sql_query)  # Ensure this returns the result of executing the SQL query
    
    # Step 3: Pass the necessary inputs to the final chain and format the output to include both SQL query and result
    final_response = execute_sql_chain.invoke({"question": question, "query": sql_query, "result": sql_result})
    
    # Format the final answer to include both the SQL query and its result
    final_answer = f"SQL Query: {sql_query}\n Answer: {final_response}"
    return final_answer


In [102]:
# Example invocation
question = "what is the volume of model 1?"
final_answer = execute_combined_chain(question)
print(final_answer)

SQL Query: SELECT "Daily_Volume" 
FROM models 
WHERE "Model_Name" = 'Model 1';
 Answer: The volume of Model 1 is not clear from the SQL result provided. The query returned a list of daily volumes for Model 1, but it seems to be a list of multiple values rather than a single volume. To determine the volume of Model 1, we would need to calculate the average or total volume from the list of daily volumes provided in the SQL result.


### Further Fine tunning the model

In [83]:
db = SQLDatabase.from_uri("sqlite:///dataset.db")
print(db.dialect)
print(db.get_usable_table_names())

# Define the SQL query for the view
view_query = """
CREATE VIEW model_metrics_view AS
SELECT
    MODEL_ID,
    CASE 
        WHEN "Model_Type" IN ('Multi-Class', 'Classification') 
            THEN json_extract("Performance_Metrics", '$[0]')
    END AS recall,
    CASE 
        WHEN "Model_Type" IN ('Multi-Class', 'Classification') 
            THEN json_extract("Performance_Metrics", '$[1]')
    END AS precision,
    CASE 
        WHEN "Model_Type" IN ('Multi-Class', 'Classification') 
            THEN json_extract("Performance_Metrics", '$[2]')
    END AS accuracy,
    CASE
        WHEN "Model_type" IN ("Regression")
             THEN Performance_Metrics
    END AS MAE
FROM
    models;
"""

# Execute the query to create the view
db.run(view_query)


sqlite
['models']


''

In [94]:
# Updated SQL Prompt
SQl_prompt_updated = PromptTemplate.from_template(
    """You are a SQLite expert. Given an input question, create a syntactically correct SQLite query to run, to answer the input question.
Remember, your query should aim to answer the question with a single SQL statement and limit the results appropriately. You can order the results to return the most informative data in the database.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
Never query for all columns from a table. Only query the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Be careful not to query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use the date('now') function to get the current date if the question involves "today".
Only use the following tables:

{table_info}.

Question: {input}
Provide up to {top_k} SQL variations.

For questions related to performance metrics or accuracy, MAE or recall, you can consider joining the 'models' table with the 'model_metrics_view' view on the MODEL_ID column."""
)


In [95]:
write_query = create_sql_query_chain(prompt=SQl_prompt_updated, llm=llm, db=db)
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | execute_query
chain.invoke({"question": "Which model has the best accuracy"})

'Error: (sqlite3.OperationalError) near "1.": syntax error\n[SQL: 1. \n```sql\nSELECT "Model_Name", MAX(CAST(JSON_EXTRACT("Performance_Metrics", \'$[2]\') AS REAL)) AS "Best_Accuracy"\nFROM models\nGROUP BY "Model_Name"\nORDER BY "Best_Accuracy" DESC\nLIMIT 1;\n```\n\n2. \n```sql\nSELECT "Model_Name", MAX(CAST(JSON_EXTRACT("Performance_Metrics", \'$[2]\') AS REAL)) AS "Best_Accuracy"\nFROM models\nORDER BY "Best_Accuracy" DESC\nLIMIT 1;\n```\n\n3. \n```sql\nSELECT "Model_Name", MAX(CAST(JSON_EXTRACT("Performance_Metrics", \'$[2]\') AS REAL)) AS "Best_Accuracy"\nFROM models\nGROUP BY "Model_Name"\nORDER BY "Best_Accuracy" DESC\nLIMIT 1;\n```\n\n4. \n```sql\nSELECT "Model_Name", MAX(CAST(JSON_EXTRACT("Performance_Metrics", \'$[2]\') AS REAL)) AS "Best_Accuracy"\nFROM models\nORDER BY "Best_Accuracy" DESC\nLIMIT 1;\n```\n\n5. \n```sql\nSELECT "Model_Name", MAX(CAST(JSON_EXTRACT("Performance_Metrics", \'$[2]\') AS REAL)) AS "Best_Accuracy"\nFROM models\nGROUP BY "Model_Name"\nORDER BY "Bes